## Some charts about COVID-19 
Provided daily via Telegram channel: https://t.me/covid19charts. You can subscribe and receive automatic updates.

Some basics charts about COVID-19 from https://github.com/CSSEGISandData/COVID-19.git
**Cumulative, daily and percentage** charts on **Region/Country** with more than **500 Confirmed**. 

Code available at https://github.com/andreaaizza/covid-19-basic-charts

In [ ]:
%reset
import pandas as pd
df_c = pd.read_csv(
    "./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
df_c=df_c.drop(columns=['Lat','Long'])
df_c=df_c.groupby('Country/Region').sum()

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib qt
%matplotlib inline

In [ ]:
# load Confirmed to filter
df_c = pd.read_csv(
    "./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
df_c=df_c.drop(columns=['Lat','Long'])
df_c=df_c.groupby('Country/Region').sum()

# get only >500
last_col=df_c.columns[len(df_c.columns)-1]
significant_ones=df_c[df_c[last_col]>500].index

# load all files
df_suff=['Confirmed','Deaths','Recovered']
df_suff_without_confirmed=['Deaths','Recovered']
df=pd.DataFrame()
for s in df_suff:
    d=pd.read_csv(
    "./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-"+s+".csv")
    d=d.drop(columns=['Lat','Long'])
    d=d.groupby('Country/Region').sum()
    d['Type']=s
    df=df.append(d)
# set multiindex
df=df.set_index('Type', append=True)

In [ ]:
plt.figure(figsize=(14, 6), dpi= 80, facecolor='w', edgecolor='k')
for l in significant_ones:
    if l!='Mainland China' and l!='Others':
        plt.plot(df_c[df_c.index==l].iloc[0], label=l)

plt.title('Cumulative Confirmed only')
plt.xticks(rotation=90)
plt.legend()
plt.grid()


In [ ]:
def plotCounties(title, f): 
    plt_size=len(significant_ones)
    fig, ax=plt.subplots(nrows=plt_size,ncols=1,figsize=(14, 14), dpi=80, facecolor='w', edgecolor='k')
    fig.suptitle(title)

    for i in range(0,plt_size): 
        country=significant_ones[i]
        # plot country
        for t in df_suff:
            f(df, ax[i], t, country)
            ax[i].set_title(country)
            ax[i].grid()
            ax[i].legend()
            if i==plt_size-1:
                plt.xticks(df.columns, rotation=90)
            else:
                ax[i].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

In [ ]:
def p_cum(df, ax, t, country):
    ax.plot(df[
            (df.index.get_level_values('Type')==t) & 
            (df.index.get_level_values('Country/Region')==country)].iloc[0],'.-', label=t)
    
plotCounties('Cumulative', p_cum)

In [ ]:
def p_daily(df, ax, t, country):
    data=df[
            (df.index.get_level_values('Type')==t) & 
            (df.index.get_level_values('Country/Region')==country)].iloc[0]
    ax.plot(data-data.shift(1),'.-', label=t)
    
plotCounties('Daily', p_daily)

In [ ]:
def p_ppcc(df, ax, t, country):
    data_confirmed=df[
        (df.index.get_level_values('Type')=='Confirmed') & 
        (df.index.get_level_values('Country/Region')==country)].iloc[0]

    data=df[
        (df.index.get_level_values('Type')==t) & 
        (df.index.get_level_values('Country/Region')==country)].iloc[0]
    data_delta=data-data.shift(1)
    ax.plot(data_delta/data_confirmed,'.-', label=t)    
    
plotCounties('Daily as percentage of Confirmed', p_ppcc)